In [47]:
import sys
sys.path.insert(0, '/home/jovyan/work/Molecular_Properties/model-pipeline/src')
import pandas as pd
import warnings
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from model_construction import prepare_data_split, run_model, show_varimp
from bearinmind_pipeline import data_preprocessing
import gc
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import os
import lightgbm as lgb
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)


In [2]:
# !kaggle competitions download -c ieee-fraud-detection
# !unzip \*.zip
# !rm *.zip
# !chmod 600 *.csv

In [3]:
gc.collect()

55

In [4]:
df_train_id = pd.read_csv('train_identity.csv')
df_test_id = pd.read_csv('test_identity.csv')
df_train_txn = pd.read_csv('train_transaction.csv')
df_test_txn = pd.read_csv('test_transaction.csv')

In [5]:
df_train_txn = df_train_txn.merge(df_train_id, how = 'left', on  = 'TransactionID')
df_test_txn = df_test_txn.merge(df_test_id, how = 'left', on  = 'TransactionID')

In [6]:
df_train_txn['isFraud'].value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [7]:
#df_train_txn = df_train_txn[df_train_txn.columns.drop(list(df_train_txn.filter(regex='M[0-9]')))]
#df_test_txn = df_test_txn[df_test_txn.columns.drop(list(df_test_txn.filter(regex='M[0-9]')))]

In [8]:
df_train_txn.dtypes[df_train_txn.dtypes=='object']

ProductCD        object
card4            object
card6            object
P_emaildomain    object
R_emaildomain    object
M1               object
M2               object
M3               object
M4               object
M5               object
M6               object
M7               object
M8               object
M9               object
id_12            object
id_15            object
id_16            object
id_23            object
id_27            object
id_28            object
id_29            object
id_30            object
id_31            object
id_33            object
id_34            object
id_35            object
id_36            object
id_37            object
id_38            object
DeviceType       object
DeviceInfo       object
dtype: object

In [9]:
#df_train_txn['M9'].describe()

In [10]:
df_train_txn['DeviceInfo'] = np.where(~df_train_txn.DeviceInfo.isin(['Windows', 'iOS Device', 'MacOS']), 'Android', df_train_txn.DeviceInfo)

In [11]:
df_test_txn['DeviceInfo'] = np.where(~df_test_txn.DeviceInfo.isin(['Windows', 'iOS Device', 'MacOS']), 'Android', df_test_txn.DeviceInfo)

In [12]:
df_train_txn.shape

(590540, 434)

In [13]:
def create_stats_features(df, nominator, groupby):
    df[f'{nominator}_by_{groupby}_std'] = df[nominator]/df.groupby(groupby)[nominator].transform('std')
    df[f'{nominator}_by_{groupby}_mean'] = df[nominator]/df.groupby(groupby)[nominator].transform('mean')
    df[f'{nominator}_by_{groupby}_median'] = df[nominator]/df.groupby(groupby)[nominator].transform('median')
    df[f'{nominator}_by_{groupby}_max'] = df[nominator]/df.groupby(groupby)[nominator].transform('max')
    df[f'{nominator}_by_{groupby}_min'] = df[nominator]/df.groupby(groupby)[nominator].transform('min')
    
    return df

In [14]:
#df_train_txn = create_stats_features(df_train_txn, 'TransactionAmt', 'card6')
#df_test_txn = create_stats_features(df_test_txn, 'TransactionAmt', 'card6')

In [15]:
df_train_txn.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.500,W,13926,nan,150.000,discover,142.000,credit,315.000,87.000,19.000,nan,NaN,NaN,1.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,2.000,0.000,1.000,1.000,14.000,nan,13.000,nan,nan,nan,nan,nan,nan,13.000,13.000,nan,nan,nan,0.000,T,T,T,M2,F,T,NaN,NaN,NaN,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,0.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,1.000,1.000,0.000,0.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,1.000,1.000,0.000,0.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,117.000,0.000,0.000,0.000,0.000,0.000,117.000,0.000,0.000,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,0.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,117.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,117.000,0.000,0.00

In [16]:
# parse Email column
df_train_txn['R_emaildomain'] = df_train_txn['R_emaildomain'].str.split('.').str.get(0)
df_test_txn['P_emaildomain'] = df_test_txn['P_emaildomain'].str.split('.').str.get(0)
df_test_txn['R_emaildomain'] = df_test_txn['R_emaildomain'].str.split('.').str.get(0)
df_train_txn['P_emaildomain'] = df_train_txn['P_emaildomain'].str.split('.').str.get(0)

In [17]:
#parse id 30 column
df_train_txn['id_30'] = df_train_txn['id_30'].str.split(' ').str.get(0)
df_test_txn['id_30'] = df_test_txn['id_30'].str.split(' ').str.get(0)

In [18]:
#parse id 31 column
df_train_txn['id_31'] = df_train_txn['id_31'].str.split(' ').str.get(0)
df_test_txn['id_31'] = df_test_txn['id_31'].str.split(' ').str.get(0)

In [19]:
df_train_txn['id_31'] = df_train_txn['id_31'].str.split('/').str.get(0)
df_test_txn['id_31'] = df_test_txn['id_31'].str.split('/').str.get(0)

In [20]:
df_train_txn['id_31'] = df_train_txn['id_31'].str.lower()
df_test_txn['id_31'] = df_test_txn['id_31'].str.lower()

In [21]:
df_train_txn['id_31'] = np.where(df_train_txn.id_31.isin(['mozilla']), 'firefox', df_train_txn.id_31)
df_test_txn['id_31'] = np.where(df_test_txn.id_31.isin(['mozilla']), 'firefox', df_test_txn.id_31)

In [22]:
#parse id 33 resolution
df_train_txn['horizontal_pixels'] = df_train_txn['id_33'].str.split('x').str.get(0)
df_test_txn['horizontal_pixels'] = df_test_txn['id_33'].str.split('x').str.get(0)
df_train_txn['vertical_pixels'] = df_train_txn['id_33'].str.split('x').str.get(1)
df_test_txn['vertical_pixels'] = df_test_txn['id_33'].str.split('x').str.get(1)

In [23]:
df_train_txn['horizontal_pixels'] = pd.to_numeric(df_train_txn['horizontal_pixels'])
df_test_txn['horizontal_pixels'] = pd.to_numeric(df_test_txn['horizontal_pixels'])
df_train_txn['vertical_pixels'] = pd.to_numeric(df_train_txn['vertical_pixels'])
df_test_txn['vertical_pixels'] = pd.to_numeric(df_test_txn['vertical_pixels'])

In [24]:
df_train_txn = df_train_txn.drop('id_33', axis = 1)
df_test_txn = df_test_txn.drop('id_33', axis = 1)

In [25]:
#at_cols = ['card1', 'card2', 'card3', 'card5', 'card6', 'addr1', 'addr2']

In [26]:
encode_cols = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'DeviceType', 'DeviceInfo', 'id_12', 'M1', 'M2', 'M3', 'M4', 'M5',  'M6', 'M7', 'M8', 'M9',       
              'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

In [27]:
#encode_categorical(df_train_txn, df_test_txn, cat_cols)
#df_train_txn = df_train_txn.drop(cat_cols, axis = 1)

In [28]:
cols = list(df_train_txn.columns.values)

In [29]:
cols.remove('isFraud')

In [30]:
df_test_txn = df_test_txn[cols]

In [31]:
df_test_txn.dtypes 

TransactionID          int64
TransactionDT          int64
TransactionAmt       float64
ProductCD             object
card1                  int64
card2                float64
card3                float64
card4                 object
card5                float64
card6                 object
addr1                float64
addr2                float64
dist1                float64
dist2                float64
P_emaildomain         object
R_emaildomain         object
C1                   float64
C2                   float64
C3                   float64
C4                   float64
C5                   float64
C6                   float64
C7                   float64
C8                   float64
C9                   float64
C10                  float64
C11                  float64
C12                  float64
C13                  float64
C14                  float64
                      ...   
id_12                 object
id_13                float64
id_14                float64
id_15         

In [32]:
#df_train_txn['M1'].value_counts()

In [33]:
for f in encode_cols:
    print(f'colums {f} is being encoded')
    lbl = LabelEncoder()
    lbl.fit(list(df_train_txn[f].values) + list(df_test_txn[f].values))
    df_train_txn[f] = lbl.transform(list(df_train_txn[f].values))
    df_test_txn[f] = lbl.transform(list(df_test_txn[f].values))

colums ProductCD is being encoded
colums card4 is being encoded
colums card6 is being encoded
colums P_emaildomain is being encoded
colums R_emaildomain is being encoded
colums DeviceType is being encoded
colums DeviceInfo is being encoded
colums id_12 is being encoded
colums M1 is being encoded
colums M2 is being encoded
colums M3 is being encoded
colums M4 is being encoded
colums M5 is being encoded
colums M6 is being encoded
colums M7 is being encoded
colums M8 is being encoded
colums M9 is being encoded
colums id_15 is being encoded
colums id_16 is being encoded
colums id_23 is being encoded
colums id_27 is being encoded
colums id_28 is being encoded
colums id_29 is being encoded
colums id_30 is being encoded
colums id_31 is being encoded
colums id_34 is being encoded
colums id_35 is being encoded
colums id_36 is being encoded
colums id_37 is being encoded
colums id_38 is being encoded


In [34]:
df_train_txn.shape

(590540, 435)

In [35]:
def prepare_data_split(df_train, df_test, target, rem_cols, useVarImp = False):
    if useVarImp:
        varImp = pd.read_csv(f'varImp/lgbm_importance.csv')
        varImp = varImp[varImp.Value >= 100]
        varImpCount = varImp["Feature"].nunique() 
        varImpUnique = list(varImp['Feature'].unique())
        varImpUnique.extend(rem_cols)
        varImpUnique.extend([target])
        
        print(varImpUnique)
        
        df_train = df_train[varImpUnique]
        varImpUnique.remove(target)

        df_test = df_test[varImpUnique]

        print(f'{varImpCount} features have been chosen for modeling')
    
    else : 
        pass
    rem_cols.append(target)
    X = df_train.drop(rem_cols, axis=1)
    Y = df_train[target]
    rem_cols.remove(target)
    X_test = df_test.drop(rem_cols, axis=1)
    return [X, Y, X_test]

In [36]:
data = prepare_data_split(df_train_txn, df_test_txn, 'isFraud', rem_cols = ['TransactionID', 'TransactionDT'], useVarImp = False)

In [44]:
# LGBM
params = {
    'boosting_type':'gbdt',
    'n_estimators':1000,
    'num_leaves':128, 
    'objective':'lambdarank', 
    'colsample_bytree': 1.0, 
    'learning_rate': 0.2,
    'max_bin':255, 
    'max_depth':9, 
    'metric':'auc', 
    'min_child_samples':10,
    'min_child_weight':5, 
    'min_split_gain':0.0,
    'random_state':0, 
    'reg_alpha':0.0,
    'reg_lambda':0.0, 
    'silent':True, 
    'subsample':1.0
}

# XGBoost
params_xgb = {
    'n_estimators':500,
    'max_depth':9,
    'learning_rate':0.05,
    'subsample':0.9,
    'colsample_bytree':0.9,
    'tree_method' : 'hist',
    'nthread': 42
    #'categorical_feature': encode_cols
}

In [38]:
data[0].shape

(590540, 432)

In [39]:
submit = run_model(data[0], data[1], data[2], 5, problem_type = 'classification', model_type = 'LGBM', metric_func = roc_auc_score, get_probab = True, save_varimp = True, params = params)

Fold: 0
The valuation metric for the fold 0 is 0.965261863031469
Fold: 1
The valuation metric for the fold 1 is 0.9647577511329167
Fold: 2
The valuation metric for the fold 2 is 0.9613237609628096
The average score accross the folds is 0.9637811250423983


In [40]:
submit_pred= pd.DataFrame(submit[0])
submit_pred.columns = ['isFraud']
sub = pd.concat([df_test_txn['TransactionID'], submit_pred], axis = 1)

In [42]:
sub.to_csv('submission_model_fraud_lightgbm.csv', index = False)

In [43]:
!kaggle competitions submit -c ieee-fraud-detection -f submission_model_fraud_lightgbm.csv -m "Working submission my lightgbm"

100%|██████████████████████████████████████| 14.4M/14.4M [00:05<00:00, 2.57MB/s]
Successfully submitted to IEEE-CIS Fraud Detection